In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import math

D:\anaconda\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [4]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = 'Dataset/Train'
valid_path = 'Dataset/Test'

In [5]:

# add preprocessing layer to the front of VGG
inception_resnet_v2 = InceptionResNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [7]:
# don't train existing weights
for layer in inception_resnet_v2.layers:
    layer.trainable=False

In [9]:
# useful for getting number of classes
folders = glob('Dataset/Train/*')  
folders

['Dataset/Train\\Biden',
 'Dataset/Train\\Dhruv',
 'Dataset/Train\\Elon',
 'Dataset/Train\\Obama']

In [10]:
# our layers - you can add more if you want
x = Flatten()(inception_resnet_v2.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folde rs), activation='softmax')(x)

# create a model object
model = Model(inputs=inception_resnet_v2.input, outputs=prediction)

# view the structure of the model
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_204 (BatchN (None, 111, 111, 32) 96          conv2d_204[0][0]                 
__________________________________________________________________________________________________
activation_204 (Activation)     (None, 111, 111, 32) 0           batch_normalization_204[0][0]    
____________________________________________________________________________________________

__________________________________________________________________________________________________
activation_240 (Activation)     (None, 25, 25, 32)   0           batch_normalization_240[0][0]    
__________________________________________________________________________________________________
activation_242 (Activation)     (None, 25, 25, 32)   0           batch_normalization_242[0][0]    
__________________________________________________________________________________________________
activation_245 (Activation)     (None, 25, 25, 64)   0           batch_normalization_245[0][0]    
__________________________________________________________________________________________________
block35_5_mixed (Concatenate)   (None, 25, 25, 128)  0           activation_240[0][0]             
                                                                 activation_242[0][0]             
                                                                 activation_245[0][0]             
__________

block17_2 (Lambda)              (None, 12, 12, 1088) 0           block17_1_ac[0][0]               
                                                                 block17_2_conv[0][0]             
__________________________________________________________________________________________________
block17_2_ac (Activation)       (None, 12, 12, 1088) 0           block17_2[0][0]                  
__________________________________________________________________________________________________
conv2d_289 (Conv2D)             (None, 12, 12, 128)  139264      block17_2_ac[0][0]               
__________________________________________________________________________________________________
batch_normalization_289 (BatchN (None, 12, 12, 128)  384         conv2d_289[0][0]                 
__________________________________________________________________________________________________
activation_289 (Activation)     (None, 12, 12, 128)  0           batch_normalization_289[0][0]    
__________

conv2d_346 (Conv2D)             (None, 12, 12, 160)  143360      activation_345[0][0]             
__________________________________________________________________________________________________
batch_normalization_346 (BatchN (None, 12, 12, 160)  480         conv2d_346[0][0]                 
__________________________________________________________________________________________________
activation_346 (Activation)     (None, 12, 12, 160)  0           batch_normalization_346[0][0]    
__________________________________________________________________________________________________
conv2d_344 (Conv2D)             (None, 12, 12, 192)  208896      block17_16_ac[0][0]              
__________________________________________________________________________________________________
conv2d_347 (Conv2D)             (None, 12, 12, 192)  215040      activation_346[0][0]             
__________________________________________________________________________________________________
batch_norm

batch_normalization_387 (BatchN (None, 5, 5, 192)    576         conv2d_387[0][0]                 
__________________________________________________________________________________________________
batch_normalization_390 (BatchN (None, 5, 5, 256)    768         conv2d_390[0][0]                 
__________________________________________________________________________________________________
activation_387 (Activation)     (None, 5, 5, 192)    0           batch_normalization_387[0][0]    
__________________________________________________________________________________________________
activation_390 (Activation)     (None, 5, 5, 256)    0           batch_normalization_390[0][0]    
__________________________________________________________________________________________________
block8_6_mixed (Concatenate)    (None, 5, 5, 448)    0           activation_387[0][0]             
                                                                 activation_390[0][0]             
__________

In [11]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [12]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [15]:
training_set = train_datagen.flow_from_directory('Dataset/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Dataset/Test',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')


Found 800 images belonging to 4 classes.
Found 400 images belonging to 2 classes.


In [22]:
batch_size=16
training_size=800
validation_size=400
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / batch_size))

In [27]:
steps_per_epoch = compute_steps_per_epoch(training_size)
val_steps = compute_steps_per_epoch(validation_size)
print(val_steps)
print(steps_per_epoch)

25
50


In [25]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=steps_per_epoch,
  validation_steps=val_steps
)

Epoch 1/5
49/50 [============================>.] - ETA: 2s - loss: 0.3146 - accuracy: 0.9222

ValueError: Error when checking target: expected dense_2 to have shape (4,) but got array with shape (2,)

In [1]:



# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

import tensorflow as tf

from keras.models import load_model

model.save('facefeatures_new_model.h5')



D:\anaconda\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

conv2d_49 (Conv2D)              (None, 25, 25, 32)   10240       block35_6_ac[0][0]               
__________________________________________________________________________________________________
conv2d_51 (Conv2D)              (None, 25, 25, 32)   9216        activation_50[0][0]              
__________________________________________________________________________________________________
conv2d_54 (Conv2D)              (None, 25, 25, 64)   27648       activation_53[0][0]              
__________________________________________________________________________________________________
batch_normalization_49 (BatchNo (None, 25, 25, 32)   96          conv2d_49[0][0]                  
__________________________________________________________________________________________________
batch_normalization_51 (BatchNo (None, 25, 25, 32)   96          conv2d_51[0][0]                  
__________________________________________________________________________________________________
batch_norm

__________________________________________________________________________________________________
block17_5_ac (Activation)       (None, 12, 12, 1088) 0           block17_5[0][0]                  
__________________________________________________________________________________________________
conv2d_98 (Conv2D)              (None, 12, 12, 128)  139264      block17_5_ac[0][0]               
__________________________________________________________________________________________________
batch_normalization_98 (BatchNo (None, 12, 12, 128)  384         conv2d_98[0][0]                  
__________________________________________________________________________________________________
activation_98 (Activation)      (None, 12, 12, 128)  0           batch_normalization_98[0][0]     
__________________________________________________________________________________________________
conv2d_99 (Conv2D)              (None, 12, 12, 160)  143360      activation_98[0][0]              
__________

block17_19_conv (Conv2D)        (None, 12, 12, 1088) 418880      block17_19_mixed[0][0]           
__________________________________________________________________________________________________
block17_19 (Lambda)             (None, 12, 12, 1088) 0           block17_18_ac[0][0]              
                                                                 block17_19_conv[0][0]            
__________________________________________________________________________________________________
block17_19_ac (Activation)      (None, 12, 12, 1088) 0           block17_19[0][0]                 
__________________________________________________________________________________________________
conv2d_154 (Conv2D)             (None, 12, 12, 128)  139264      block17_19_ac[0][0]              
__________________________________________________________________________________________________
batch_normalization_154 (BatchN (None, 12, 12, 128)  384         conv2d_154[0][0]                 
__________

batch_normalization_195 (BatchN (None, 5, 5, 256)    768         conv2d_195[0][0]                 
__________________________________________________________________________________________________
activation_192 (Activation)     (None, 5, 5, 192)    0           batch_normalization_192[0][0]    
__________________________________________________________________________________________________
activation_195 (Activation)     (None, 5, 5, 256)    0           batch_normalization_195[0][0]    
__________________________________________________________________________________________________
block8_8_mixed (Concatenate)    (None, 5, 5, 448)    0           activation_192[0][0]             
                                                                 activation_195[0][0]             
__________________________________________________________________________________________________
block8_8_conv (Conv2D)          (None, 5, 5, 2080)   933920      block8_8_mixed[0][0]             
__________

flatten_1 (Flatten)             (None, 38400)        0           conv_7b_ac[0][0]                 
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 4)            153604      flatten_1[0][0]                  
Total params: 54,490,340
Trainable params: 153,604
Non-trainable params: 54,336,736
__________________________________________________________________________________________________
Found 800 images belonging to 4 classes.
Found 400 images belonging to 2 classes.
Epoch 1/5
49/50 [============================>.] - ETA: 5s - loss: 0.1723 - accuracy: 0.9541 

ValueError: Error when checking target: expected dense_1 to have shape (4,) but got array with shape (2,)